# Q and A on Scientific Journal

<div class="align-center">
  <a href="https://getindexify.ai/"><img src="https://getindexify.ai/Indexify_Logo_Wordmark.svg" width="145"></a>
  <a href="https://discord.com/invite/kF8UZACA7r"><img src="https://raw.githubusercontent.com/rishiraj/random/main/Discord%20button.png" width="145"></a><br>
  Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/tensorlakeai/indexify">Github</a></i> ⭐
</div>

In this notebook we show text extraction from a scientific paper. We are going to chunk and embed the text after text extraction. We show Langchain based extraction.

## **Setup**

In [ ]:
%pip install indexify indexify-extractor-sdk indexify-langchain langchain langchain-openai

# Download Indexify Server
!curl https://getindexify.ai | sh

# Install Poppler (required for PDF extraction)
# You can use brew on MacOS.
!sudo apt-get install -y poppler-utils

# Download Extractors
!indexify-extractor download tensorlake/chunk-extractor
!indexify-extractor download tensorlake/minilm-l6
!indexify-extractor download tensorlake/pdf-extractor

After installing the necessary libraries, download the server, and the extractors, you need to restart the runtime. Then, you have to run Indexify Server with the Extractors.

Open 2 terminals and run the following commands:

```bash
# Terminal 1
./indexify server -d

# Terminal 2
indexify-extractor join-server
```

## **Create Extraction Graph**

In [ ]:
from indexify import IndexifyClient, ExtractionGraph
client = IndexifyClient()

Create an Extraction Graph with policies to:
1. Get texts and contents out of the PDF.
2. Create chunks from the text.
3. Generate embeddings from the chunks.

In [ ]:
extraction_graph_spec = """
name: 'sciknowledgebase'
extraction_policies:
  - extractor: 'tensorlake/pdf-extractor'
    name: 'pdfextractor'
  - extractor: 'tensorlake/chunk-extractor'
    name: 'chunks'
    content_source: 'pdfextractor'
    input_params:
      chunk_size: 512
      overlap: 150
  - extractor: 'tensorlake/bge-base-en'
    name: 'bgeembedding'
    content_source: 'chunks'
"""
extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
client.create_extraction_graph(extraction_graph)  

## **Upload a PDF File**

In [ ]:
import requests
req = requests.get("https://arxiv.org/pdf/2310.16944.pdf")

with open('2310.16944.pdf','wb') as f:
    f.write(req.content)

In [ ]:
content_id = client.upload_file("sciknowledgebase", path="2310.16944.pdf")
client.wait_for_extraction(content_id)  

## **What is happening behind the scenes**

Indexify is designed to seamlessly respond to ingestion events by assessing all existing policies and triggering the necessary extractors for extraction. Once the PDF extractor completes the process of extracting texts, bytes, and JSONs from the document, it automatically initiates the embedding extractor to chunk the content, extract embeddings, and populate an index.

With Indexify, you have the ability to upload hundreds of PDF files simultaneously, and the platform will efficiently handle the extraction and indexing of the contents without requiring manual intervention. To expedite the extraction process, you can deploy multiple instances of the extractors, and Indexify's built-in scheduler will transparently distribute the workload among them, ensuring optimal performance and efficiency.

## **Perform RAG**
Initialize the Langchain Retriever.

In [ ]:
from indexify_langchain import IndexifyRetriever
params = {"name": "sciknowledgebase.bgeembedding.embedding", "top_k": 15}
retriever = IndexifyRetriever(client=client, params=params)

Now create a chain to prompt OpenAI with data retrieved from Indexify to create a simple Q&A bot

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

Now ask any question related to the ingested PDF document

In [ ]:
chain.invoke("What are the results on MT-Bench?")
# Results on MT-Bench show that ZEPHYR-7B surpasses LLAMA 2-CHAT-70B, the best open-access RLHF-based model.

In [ ]:
chain.invoke("what prompts are they using? Show all the prompts ")